# Topics

## 1. DFT and Power Spectrum
## 2. Parseval's Theorem



## In what follows:
## tone_data: y_k
## ft = np.fft(tone_data): Y_n

## A[0] contains the zero-frequency term (the sum of the signal), which is always purely real for real inputs. Then A[1:n/2-1] contains the positive-frequency terms, and A[n/2:] contains the negative-frequency terms,  in order of decreasingly negative frequency. For an even number of input points, A[n/2] represents both positive and negative Nyquist frequency, and is also purely real for real input.

Modified from: http://docs.scipy.org/doc/numpy/reference/routines.fft.html


In [1]:
%matplotlib inline

import matplotlib.pyplot as plt
import numpy as np
from scipy.io.wavfile import read, write
import os
import subprocess as sub
from IPython.lib.display import Audio

from pdb import set_trace


In [2]:
import winsound
import time


def playsound(playrate, numpyArray):
    write('temp.wav', playrate, numpyArray)
    winsound.PlaySound('temp.wav', winsound.SND_ASYNC)
    duration = len(numpyArray)/playrate
    time.sleep(duration)
    os.remove('temp.wav')

ModuleNotFoundError: No module named 'winsound'

## Breakout  
- ### Write a function get_music(music_file, start_time, end_time) and it returns sample_rate, time_pts, sound_array

In [ ]:
def get_music(music_file, start_time, end_time):
    rate, barry_white = read(music_file)
    duration = end_time - start_time
    time_pts = np.linspace(0, duration, duration*rate)
    start_point = start_time*rate
    end_point = end_time*rate
    new_barry_white = barry_white[start_point: end_point]
    return rate, time_pts, new_barry_white
    

In [ ]:
start_time = 1
end_time = 5


sample_rate, time_pts, shortBW = get_music("BarryWhite30sec.wav", start_time, end_time)
print("sample_rate", sample_rate)

playsound(sample_rate, shortBW)

# plotsound(time_pts, shortBW, s_lim = 3000)

# check if sound file plays with Audio -- if necessary
# Audio(shortBW, rate = sample_rate)


In [ ]:
# can we get Barry White back?

# Here there are indeed real and imaginary parts for the FT.
ft, freq = plot_fourier(sample_rate, shortBW, freq_lim = 5000.)
BW_rec = np.int16(signal_rec(ft, filt = None))
playsound(BW_rec, vol = 10)

fig = plt.figure(figsize = (10, 8))
# Note here I'm passing fig as an argument -- so that the original array and the 
# recovered array can be plotted on the same figure.
plotsound(time_pts, shortBW, fig = fig, s_lim = 3000, plot_style = 'b-')
plotsound(time_pts, BW_rec, fig = fig, s_lim = 3000, plot_style = 'gx')
plt.show()

## Breakout Exercise: Low-pass filter
- ### Remove Fourier components with frequencies > 1000 Hz from Barry White; call the sound array with high frequency component removed BW_base.
- ### Play the original shortBW and then BW_base
- ### Plot shortBW and BW_base on the same figure, for time between [0, 0.02] sec.

In [ ]:
def gen_tone(f, duration, sample_rate = 44100, amp = 2**13, play_sound = False):
    time_pts = np.linspace(0, duration, duration*sample_rate)
    tone_data = amp*np.sin(np.pi*2*f*time_pts)
    return tone_data

def plot_fourier(sample_rate, signal, freq_lim = 1000.):
    ft = np.fft.fft(np.float(signal))
    freq = np.fft.fftfreq(signal.shape[0], d = 1/sample_rate)
    
    plt.figure()
    plt.title("Real Part of Inverse FT")
    plt.plot(freq, ft.real)
    plt.xlim([-freq_lim, freq_lim])
    
    plt.figure()
    plt.title("Imaginary Part of Inverse FT")
    plt.plot(freq, ft.imag)
    plt.xlim([-freq_lim, freq_lim])
    
    return ft, freq

## Breakout Exercise: High-pass filter
- ### Remove Fourier components with frequencies < 1000 Hz from Barry White; call the sound array with low frequency component removed BW_base.
- ### Play the original shortBW and then BW_hi_pitch
- ### Plot shortBW and BW_base on the same figure, for time between [0, 0.02] sec.

In [ ]:
# More base!
BW_base = np.float64(BW_base)
BW_hi_pitch = np.float64(BW_hi_pitch)

# BW_base *= 4/np.sqrt(17.)
# BW_hi_pitch *= 1/np.sqrt(17.)

BW_base *= 2/np.sqrt(5.)
BW_hi_pitch *= 1/np.sqrt(5.)


Ultra_BW = np.int16(BW_base + BW_hi_pitch)

playsound(shortBW, vol = 10)
playsound(Ultra_BW, vol = 10)

## Parseval's Theorem
### (Show slides first!)

In [ ]:
%matplotlib inline
'''

Parseval's Theorem and Conservation of Energy.

'''

import matplotlib.pyplot as plt

sample_rate = 44100.

duration = 2.


f = 440.
time_pts = np.linspace(0, duration, duration*sample_rate)

amp = 2**14
tone_data = np.int16(amp*np.sin(np.pi*2*f*time_pts))

# playsound(tone_data, vol = 0.5)


ft = np.fft.fft(tone_data)
freq = np.fft.fftfreq(tone_data.shape[0], d = 1/sample_rate)


# Power spectrum
# checking Persarvel's Theorem
# B/c of the way np.fft is defined, the power spectrum is (|ft|/N)^2
N = len(tone_data)
print('N =', N)
pwr = (ft * ft.conj())/N**2

if pwr.imag.max() > 1e-15:
    raise KeyboardInterrupt('Power is not real...something is wrong!!')

plt.figure()
plt.plot(freq, pwr.real, 'k-')
plt.xlim([-1000, 1000])


print("Power spectrum summed: {:g}".format(pwr.sum()))
print('Sum of data squared divided by N: {:g}'.format((tone_data**2).sum()/N))
print('Sum of data squared divided by N: {:g}'.format((tone_data.astype(np.float)**2).sum()/N))



In [ ]:
'''

Scipy's ready-made way of computing the (one-sided) power spectral density.


It probably ends at Nyquist Frequency (since going further doesn't gain new information) -- if so that that's another
advantage of using periodogram: it gets rid of four-fold redundancy, 1) +ve and -ve freq, 2) on the two sides of
the Nyquist frequency.


'''


from scipy import signal

# note the powers at the negative frequencies have been added to the powers of the corresponding 
# positive freqeuncies.
f, psd = signal.periodogram(tone_data, sample_rate)
df = 1./duration
plt.figure()
plt.plot(f, psd)
plt.xlim([0, 1000])
plt.show()
# This should be N/2 + 1 (b/c of the zero-frequency term)
print('len(psd)', len(psd))
# sum of psd * df
print('Power spectral density integrated: {:g}'.format((psd*df).sum()))

plt.show()

## Breakout: Two-frequency signal and PSD 

- ### Turn the above into a function the plot_psd(sound, sample_rate = 44100, duration = 0., freq_lim = 1000., pwr_lim = 3e7); it should return the power spectrum (psd)
- ### Generate a sigal that has two frequency components (fundamental frequencey (1st harmonic) + 2nd harmonic)
- ### Apply this funcion on a 2-freq sound and see if you get what you expected.
- ### Apply this function to shortBW

In [ ]:
# psd stands for power spectrum density

def plot_psd(sound, sample_rate = 44100, duration = 0., freq_lim = 10000., pwr_lim = 3e7):
    f, psd = signal.periodogram(sound, sample_rate)
    plt.figure()
    plt.plot(f, psd)
    plt.xlim([0, 1000])
    plt.show()
    return psd



In [ ]:
# two tone sound:
f = 440.
time_pts = np.linspace(0, duration, duration*sample_rate)
amp = 2**14
tone_data1 = np.int16(amp*np.sin(np.pi*2*f*time_pts))
f2 = 880.
tone_data2 = np.int16(amp*np.sin(np.pi*2*f2*time_pts))

sound = tone_data1 + tone_data2
plot_psd(sound, duration = duration)

## Breakout -- Build a 3-band equalizer (may include this in HW06)
## Write a function: equalizer(sample_rate, sound, wt1 = None, wt2 = None, wt3 = None, vol = 1.)
- ### If any of the three weights is None, return input sound array; otherwise, do FT
- ### Split FT into three frequency ranges, < 500 Hz, [500 Hz, 1000 Hz], > 1000 Hz
- ### You can use signal_rec() to get the sound back for each of the frequency ranges
- ### You need to normalize the weights -- how would you do it?
- ### Put the three bands together according to the weight for each band, and get a new sound array.  If the weight ratio is 1:1:1, the new sound array should sound about the same as the original.
- ### Return the new sound array.
- ### Test it on the Barry White segment.

In [ ]:
def signal_rec(ft, freq, filt = None, xlo = 0, xhi = 0.02):
    import copy
    ft_filt = copy.copy(ft)
    try:
        len(filt)
    except:
        # that is, it's OK to not have a filter
        pass
    else:
        ft_filt *= filt

    tone_data_rec = np.fft.ifft(ft_filt)
    tone_data_rec = np.int16(tone_data_rec.real)

    f1 = freq[freq > 0].min() 
    dur = 1/f1
    
    time_pts = np.linspace(0, dur, len(ft))
    
    plt.figure()
    plt.title('Real Part of Inverse FT')
    plt.plot(time_pts, tone_data_rec.real, 'b-')


    plt.figure()
    plt.title('Imaginary Part of Inverse FT')
    plt.plot(time_pts, tone_data_rec.imag, 'r-')
    print('max of the imaginary part', tone_data_rec.imag.max())

    # Rejecting the imaginary part
    tone_data_rec = tone_data_rec.real

    plt.figure()
    plt.plot(time_pts, tone_data_rec, 'g')
    plt.xlim([xlo, xhi])


    plt.show()
    
    return tone_data_rec

In [ ]:
def equalizer(sample_rate, sound, wt1 = None, wt2 = None, wt3 = None, vol = 1.):
    if wt1 is None or wt2 is None or wt3 is None:
        return
    ft = np.fft.fft(sound)
    freq = np.fft.fftfreq(sound.shape[0], d = 1/sample_rate)
    
    sumwt = np.sqrt(wt1**2 + wt2**2 + wt3**2)
    filt1 = np.abs(freq) < 500.
    tone_data_rec1 = wt1 * signal_rec(ft, freq, filt = filt1) / sumwt
    
    filt2_1 = np.abs(freq) >= 500
    filt2_2 = np.abs(freq) < 1000
    filt2 = filt2_1 * filt2_2
    tone_data_rec2 = wt2 * signal_rec(ft, freq, filt = filt2) / sumwt
    
    filt3 = np.abs(freq) >= 1000
    tone_data_rec3 = wt3 * signal_rec(ft, freq, filt = filt3) / sumwt
    
    return (tone_data_rec1 + tone_data_rec2 + tone_data_rec3).astype(np.int16)

In [ ]:
sample_rate, barry = read("BarryWhite30sec.wav")

def playsound(playRate, na):
    write('temp.wav', playRate, na)
    os.system("afplay temp.wav")
    os.system("rm temp.wav")
    
playsound(sample_rate, equalizer(sample_rate, barry, 2, 1 ,1))